In [310]:
# importing required libraries
from sklearn.tree import DecisionTreeClassifier
from random import randrange
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.datasets import load_iris
import pandas as pd
import numpy as np

1. complete the following function that creates subsample of a dataset with replacement

In [311]:
# Create a random subsample from the dataset with replacement
def subsample(dataset, ratio): 
  # WRITE YOUR CODE HERE
  # IF NUMBER OF ROWS IN CURRENT DATASET IS N, THE OUTPUT DATASET SHOULD HAVE N*RATIO NUMBER OF ROWS SELECTED RANDOMLY WITH REPLACEMENT FROM THE ORIGINAL DATASET
  # INPUT AND OUTPUT DATATYPE SHOULD BE PANDAS DATAFRAME
  rows = round(len(dataset) * ratio)
  indexes = np.random.choice(len(dataset), size=rows, replace=True)
  return dataset.iloc[indexes]

2. complete the following function that creates subsample of the dataset with feature size reduced as per the given ratio

In [312]:
# Create a random selection of n features
def subsample2(dataset, ratio):
  # WRITE YOUR CODE HERE
  # THE OUTPUT DATASET SHOULD HAVE SAME NUMBER OF ROWS AS THE INPUT DATASET
  # IF NUMBER OF FEATURES IN THE INPUT DATASET IS N, THEN THE NUMBER OF FEATURES IN THE OUTPUT DATASET SHOULD BE N*ratio
  # THE INPUT AND OUTPUT DATATYPE SHOULD BE PANDAS DATAFRAME
  col = dataset.columns
  features = int(len(col) * ratio)
  col1 = np.random.choice(col, size=features, replace=False)
  return col1, dataset[col1]# RETURN 2 ITEMS, 1ST IS THE SELECTED COLUMNS (DATATYPE LIST) AND 2ND IS THE REDUCED DATAFRAME

3. Perform train test split without using sklearn (complete the following function)

In [313]:
# Splitting dataset into train and test
def split_train_test(dataset, test_size):
  # WRITE YOUR CODE HERE
  # DATASET IS THE INPUT DATAFRAME
  # TEST_SIZE IS THE SIZE OF TESTING SET IN FORM OF FRACTION BETWEEN 0 AND 1
  # OUTPUT TWO DATAFRAMES, TRAIN AND TEST DATA
  dataset = dataset.sample(frac=1)
  train_indices = len(dataset) - int(len(dataset)*test_size)
  train_data = dataset.iloc[:train_indices]
  test_data = dataset.iloc[train_indices:]

  return  train_data, test_data

4. Perform training using random forest algorithm by completing the function below

In [314]:
# Random Forest Algorithm
def random_forest_train(train, n_trees, max_depth, sample_size, n_features_ratio):
  # WRITE YOUR CODE HERE
  # TRAIN IS THE TRAINING DATASET THAT YOU GET FROM THE OUTPUT OF YOUR TRAIN TEST SPLIT
  # N_TREES IS NUMBER OF TREES IN THE FOREST
  # MAX_DEPTH IS THE MAX_DEPTH OF EACH TREE IN THE FOREST
  # SAMPLE SIZE IS THE RATIO OF ROWS TO BE SENT TO EACH TREE (VALUE BETWEEN 0 AND 1)
  # N_FEATURES_RATIO IS THE RATIO OF FEATURES TO BE SENT TO EACH TREE
  # USE SKLEARN TREES TO SIMLIFY THE EXERCISE
  result_obj = []
  for i in range(n_trees):
    data_sample = subsample(train, sample_size)
    col, data_sample2 = subsample2(data_sample.iloc[:,:-1],n_features_ratio)
    dtc = DecisionTreeClassifier(max_depth=max_depth, random_state=100)
    dtc1 = dtc.fit(data_sample[col],data_sample['target'])
    result_obj.append([dtc1, col])
  return result_obj# A LIST OF [SKLEARN TREE OBJECT, [COLS USED IN THAT TREE]] (SIZE OF OUTPUT LIST WILL BE (N_TREES*2))

5. Complete the following prediction function of random forest

In [315]:
# Make a prediction with a list of bagged trees
def random_forest_predict(test, trees):
  # WRITE YOUR CODE HERE
  # TEST IS THE TESTING DATASET THAT IS THE OUTPUT OF YOUR TRAIN TEST SPLIT FUNCTION
  # TREES ARE THE TRAINED TREES (OUTPUT OF THE RANDOM_FOREST_TRAIN FUNCTION)
  # RETURN TWO THINGS, 1 IS FINAL CLASS PREDICTIONS FOR EACH ROW IN TEST DATA, 2 IS THE PREDICTION BY EACH TREE FOR EACH ROW IN THE TEST DATA
  sub_pred = []
  complete_pred = []
  target = [0,0,0]
  k = 0
  for i in range(len(trees)):
    sub_pred.append([])
  for i in range(len(test)):
    for j in trees:
      x = test[j[1]]
      x = x.iloc[i].values.reshape(1,-1)
      pred1 = j[0].predict(x)
      target[pred1[0]] += 1
      sub_pred[k].append(pred1[0])
      k += 1
    complete_pred.append(target.index(np.max(target)))
    target = [0,0,0]
    k = 0
      

  return complete_pred, sub_pred#predictions, all_preds

In [316]:
# importing the dataset
dataset = load_iris()
df = pd.DataFrame(dataset.data)
df["target"] = dataset.target
df

,0,1,2,3,target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


In [317]:
# splitting the dataset into train and test
train, test = split_train_test(df, 0.3)

In [318]:
train = train.reset_index(drop = True)
test = test.reset_index(drop = True)

In [319]:
trees = random_forest_train(train = train, n_trees = 10, max_depth = 3, sample_size = 0.8, n_features_ratio = 0.75)

In [320]:
trees

[[DecisionTreeClassifier(max_depth=3, random_state=100),
  array([0, 3, 2], dtype=object)],
 [DecisionTreeClassifier(max_depth=3, random_state=100),
  array([0, 2, 1], dtype=object)],
 [DecisionTreeClassifier(max_depth=3, random_state=100),
  array([0, 2, 3], dtype=object)],
 [DecisionTreeClassifier(max_depth=3, random_state=100),
  array([0, 1, 2], dtype=object)],
 [DecisionTreeClassifier(max_depth=3, random_state=100),
  array([2, 3, 1], dtype=object)],
 [DecisionTreeClassifier(max_depth=3, random_state=100),
  array([1, 3, 2], dtype=object)],
 [DecisionTreeClassifier(max_depth=3, random_state=100),
  array([0, 1, 3], dtype=object)],
 [DecisionTreeClassifier(max_depth=3, random_state=100),
  array([2, 1, 3], dtype=object)],
 [DecisionTreeClassifier(max_depth=3, random_state=100),
  array([3, 0, 1], dtype=object)],
 [DecisionTreeClassifier(max_depth=3, random_state=100),
  array([2, 1, 3], dtype=object)]]

In [321]:
predictions , all_preds = random_forest_predict(test.iloc[:,:-1], trees)

In [322]:
predictions

[2,
 1,
 0,
 2,
 2,
 2,
 1,
 2,
 2,
 0,
 1,
 0,
 2,
 0,
 0,
 2,
 2,
 2,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 2,
 0,
 0,
 1,
 2,
 1,
 0,
 0,
 0,
 2,
 0,
 0]

In [323]:
all_preds

[[2,
  1,
  0,
  2,
  2,
  2,
  1,
  2,
  2,
  0,
  1,
  0,
  2,
  0,
  0,
  2,
  2,
  2,
  1,
  0,
  1,
  0,
  1,
  1,
  1,
  1,
  0,
  1,
  0,
  2,
  1,
  0,
  0,
  2,
  0,
  0,
  1,
  2,
  2,
  0,
  0,
  0,
  2,
  0,
  0],
 [2,
  1,
  0,
  2,
  2,
  2,
  1,
  2,
  2,
  0,
  1,
  0,
  2,
  0,
  0,
  2,
  2,
  2,
  1,
  0,
  1,
  0,
  1,
  1,
  1,
  1,
  0,
  1,
  0,
  2,
  2,
  0,
  0,
  2,
  0,
  0,
  1,
  2,
  2,
  0,
  0,
  0,
  2,
  0,
  0],
 [2,
  1,
  0,
  2,
  2,
  2,
  1,
  2,
  1,
  0,
  1,
  0,
  2,
  0,
  0,
  2,
  2,
  2,
  1,
  0,
  1,
  0,
  1,
  1,
  1,
  1,
  0,
  1,
  0,
  1,
  1,
  0,
  0,
  2,
  0,
  0,
  1,
  2,
  1,
  0,
  0,
  0,
  2,
  0,
  0],
 [2,
  1,
  0,
  2,
  2,
  2,
  1,
  2,
  2,
  0,
  1,
  0,
  2,
  0,
  0,
  2,
  2,
  2,
  1,
  0,
  1,
  0,
  1,
  1,
  1,
  1,
  0,
  1,
  0,
  2,
  2,
  0,
  0,
  2,
  0,
  0,
  1,
  2,
  2,
  0,
  0,
  0,
  2,
  0,
  0],
 [2,
  1,
  0,
  2,
  2,
  2,
  1,
  2,
  2,
  0,
  1,
  0,
  2,
  0,
  0,
  2,
  2,
  2,
  1,
 

In [324]:
confusion_matrix(test.iloc[:,-1], predictions)

array([[18,  0,  0],
       [ 0, 12,  0],
       [ 0,  2, 13]])

In [325]:
accuracy_score(test.iloc[:,-1], predictions)

0.9555555555555556